<table>
    <tr>
        <td><img src="./imagenes/Macc.png" width="400"/></td>
        <td>&nbsp;</td>
        <td>
            <h1 style="color:blue;text-align:left">Inteligencia Artificial</h1></td>
        <td>
            <table><tr>
            <tp><p style="font-size:150%;text-align:center">El mundo del Wumpus</p></tp>
            </tr></table>
        </td>
    </tr>
</table>

---


# Objetivo <a class="anchor" id="inicio"></a>

Vamos a entrar de lleno en el enfoque basado en agentes de la Inteligencia Artificial. Para comprenderlo mejor, debemos tomar la perspectiva de un agente que tiene que superar un reto en un minimundo. En este notebook, el minimundo es el mundo del Wumpus y uno de los objetivos es familiarizarnos con dicho entorno. Adicionalmente, implementaremos un programa de agente híbrido, el cuál incluirá un modelo lógico del mundo y una solución de objetivos basada en planeación de rutas.

Adaptado de Russell & Norvig (2016), cap. 7.

[Ir al ejercicio 1](#ej1)

## Dependencias

Al iniciar el notebook o reiniciar el kerner se pueden cargar todas las dependencias de este notebook corriendo las siguientes celdas. Este también es el lugar para instalar las dependencias que podrían hacer falta.

**Del notebook:**

In [ ]:
from logica import *
from ambientes import *
from agentes import *
from utils import *
from IPython.display import clear_output

# Secciones

En este notebook desarrollaremos los siguientes aspectos del problema:

* [El mundo del Wumpus.](#wumpus)
* [Agente híbrido.](#hibrido)
* [Representando el mundo mediante lógica proposicional.](#representacion)
* [Inferencias y queries](#queries)

# El mundo del Wumpus <a class="anchor" id="wumpus"></a>

([Volver al inicio](#inicio))

El siguiente problema se conoce como *El mundo del Wumpus* (Yob, 1975). Imagine una caverna muy oscura, representada por una rejilla de $4\times 4$ rodeada de muros. Un agente (nuestro héroe de la historia) puede entrar y salir de la caberna por la casilla (0,0) y puede percibir solamente lo que hay en cada casilla en la que se encuentre. En la caverna hay pozos muy profundos; si cae en uno de ellos, morirá. Lo peor de todo es que hay un mounstro, conocido como el Wumpus, el cual se comerá vivo al agente si éste entra a su casilla. ¿Por qué el agente entraría a un lugar como este? ¡Porque en alguna casilla de la cueva hay un montón de oro!

<img src="./imagenes/ejemplo.png" width="400">

Las siguientes son las reglas que gobiernan el mundo:

* En cualquier casilla adyacente (no diagonalmente) a un pozo se percibe una brisa; 
* En cualquier casilla adyacente (no diagonalmente) al Wumpus se percibe un hedor; 
* En la casilla donde se encuentra el oro se percibe un brillo. 
* El wumpus nunca se mueve de su casilla.

Finalmente, el agente tiene un arco y solo una flecha, con la cual puede matar al Wumpus. Cuando el agente dispara la flecha, ésta seguirá en la misma dirección del agente hasta golpear un muro o clavarse en el Wumpus, quien morirá arrojando un desgarrador grito "Aaaaaaarrrrgghhhhh".

<a class="anchor" id="ej1"></a>**Ejercicio 1:** 

([Próximo ejercicio](#ej2))

De acuerdo con la formulación de entornos hecha al comienzo del curso, ¿cuáles características considera usted que tiene este problema?

**Ayuda:** Puede usar el comando `$\checkmark$` para poner un chulito en la opción que desee marcar. 

| Opción 1 | Opción 2 |
| :---: | :---: |
| Completamente observable | Parcialmente observable|
| Agente único | Multiagente |
| Determinista | Estocástico |
| Episódico    | secuencial  |
| Estático     | dinámico    |
| Discreto     | continuo    |
| Conocido     | desconocido |

----

La **definición formal del entorno** se hace con base en la definición de las siguientes características:

* **Entorno**: Una cueva representada por una rejilla $4\times 4$ bordeada por muros. El agente siempre comienza en (0, 0) mirando a la derecha. La ubicación del Wumpus se escoge arbitrariamente de manera uniforme en casillas distintas a la inicial. Cualquier casilla distinta de la inicial puede ser un pozo con probabilidad 0.2. El oro puede estar en cualquier casilla, con probabilidad uniforme.

* **Actuadores**: El héroe puede moverse `adelante` por una casilla (no es posible moverse adelante cuando hay un muro), `voltearIzquierda` por 90º, o `voltearDerecha` por 90º. Es posible `agarrar` el oro cuando este está en la casilla ocupada por el héroe. También puede `disparar` la flecha en la dirección en que está mirando, la cual seguirá en linea recta hasta golpear un muro. Finalmente, el agente puede `salir` de la cueva, pero solo desde la casilla inicial.

* **Sensores**: El héroe percibe un `hedor` cuando llega a la casilla donde está el Wumpus o cuando llega a una de las casillas adyacentes (no diagonalmente). En las casillas adyacentes a un pozo, percibe una `brisa`. En el cuadro donde está el oro, percibe un `brillo`. Cuando se topa con un muro, percibe un `batacazo`. Finalmente, si el Wumpus muere, el heroe percibe un `grito` desde cualquier casilla.

* **Medida de desempeño**: +1000 por salir de la cueva con el oro; -1000 por caer en un pozo o ser comido por el Wumpus; -1 por cada acción y -10 por usar la flecha. El juego termina cuando el heroe muere o sale de la cueva.


**Implementación del entorno**

Inicializamos el mundo y percibimos lo que hay en la primera casilla:

In [ ]:
# Create environment
W = Wumpus(wumpus=(3,3), oro=(2,2), pozos=[(1,0), (3,1)])
state = W.reset()
print(state)

Podemos visualizar el mundo completo mediante el método `pintar_todo()`:

In [ ]:
W.pintar_todo()

Vamos a dar una serie de pasos en la oscuridad:

In [ ]:
# Create environment
W = Wumpus(wumpus=(3,3), oro=(2,2), pozos=[(1,0), (3,1)])
# Create agent
agente = Heroe()
# Create episode
episodio = Episode(environment=W,\
        agent=agente,\
        model_name='Baseline',\
        num_rounds=10)
# Include plan
camino = [(0,0), (0,1), (1,1), (2,1), (3,1)]
acciones = agente.acciones_camino(camino)
agente.plan += acciones
# Visualize
episodio.renderize()
# Presentar resumen
clear_output()
W.pintar_todo()
print(W.mensaje)

# Agente híbrido <a class="anchor" id="hibrido"></a>

([Volver al inicio](#inicio))


Nuestro heroe puede trazar rutas desde la posición en la que se en cuentra hasta una casilla objetivo (similar a lo que hace el agente basado en objetivos para salir del laberinto, el cual desarrollamos en un notebook previo). Idealmente, la casilla objetivo es segura y aún por visitar. Si no hay casillas seguras por visitar, el objetivo es la salida. 

Aún no hemos mostrado cómo el agente puede determinar si una casilla es segura, sobre lo cual volveremos en un momento.

Por ahora, las reglas que sigue el agente son las siguientes:

* Si percibe el brillo (indicando que en esa casilla está el oro), lo agarra, traza una ruta a la salida y sale de la caberna con una sonrisa en la boca y un montón de oro en el bolsillo.
* Si no percibe brillo, considera las casillas seguras que no haya visitado y se dirige a una de ellas.
* Si todas las casillas seguras ya han sido visitadas, traza una ruta a la salida y sale con las manos vacías y la cola entra las piernas.

<img src="./imagenes/program.png" width="500"/>

<a class="anchor" id="ej2"></a>**Ejercicio 2:** 

([Anterior ejercicio](#ej1)) ([Próximo ejercicio](#ej3))

Modifique el camino en la línea 11 de la celda de código anterior para que el agente llegue hasta el oro, lo agarre, luego regrese a la salida y salga. En dicha línea de código, utilice la mínima cantidad de acciones. Es decir, aproveche las reglas que ya sigue el agente.

---

<a class="anchor" id="ej3"></a>**Ejercicio 3:** 

([Anterior ejercicio](#ej2)) ([Próximo ejercicio](#ej4))

Modifique el camino en la línea 11 de la celda de código anterior para que el agente:

1. Camine hasta la esquina superior izquierda
2. Mire al este
3. Dispare la flecha,
4. Camine hasta el oro, lo agarre, regrese a la salida y salga. 

**Nota:** Tal vez deba modificar el número de rondas máximo al crear la clase `Episode`.

---

# Representación del mundo del Wumpus en lógica proposicional <a class="anchor" id="representacion"></a>

([Volver al inicio](#inicio))

Vamos a construir un modelo del Mundo del Wumpus mediante lógica proposicional. Para lograr esto, vamos a hacer que el agente sea capaz de:

* Representar las reglas del mundo del Wumpus.
* Combinar información sensorial con las reglas del mundo.
* Razonar sobre cuáles casillas adyacentes es seguro visitar.


**Letras proposicionales:**

Para comenzar la representación del Mundo del Wumpus, usaremos las siguientes letras proposicionales:

* `hedor(a, b)` es verdadero sii el agente huele un hedor en la casilla $(a,b)$.
* `brisa(a, b)` es verdadero sii el agente siente una brisa en la casilla $(a,b)$.
* `brillo(a, b)` es verdadero sii el agente ve un brillo en la casilla $(a,b)$.
* `batacazo(a, b)` es verdadero sii el agente siente un batacazo en la casilla $(a,b)$.
* `grito(a, b)` es verdadero sii el agente escucha un grito en la casilla $(a,b)$.
* `pozo(a, b)` es verdadero sii en la casilla $(a,b)$ hay un pozo.
* `wumpus(a, b)` es verdadero sii en la casilla $(a,b)$ está el Wumpus.
* `segura(a, b)` es verdadero sii la casilla $(a,b)$ es segura para transitar.

**Nota sobre las letras proposicionales:** Técnicamente hablando, las anteriores son fórmulas de la lógica de predicados. No obstante, ellas son un tipo particular de fórmulas, a saber, *fórmulas fundamentadas*. Es decir, ellas no tienen variables. Las fórmulas fundamentadas son equivalentes a letras proposicionales y podemos usar los recursos disponibles para este tipo de lógica. Usaremos mucho este truco en el resto del curso. 

**De percepciones a fórmulas**

Ahora usamos fórmulas de la lógica proposicional para representar aspectos del mundo. Observe que debemos partir de las percepciones del agente, las cuales vienen dadas mediante una lista:

['hedor'/None, 'brisa'/None, 'brillo'/None, 'batacazo'/None, 'grito'/None]

Usando el método `interp_percepto()` de la clase `Heroe` obtenemos una fórmula con la conjunción de letras proposionales que representan los perceptos:

In [ ]:
# Create environment
W = Wumpus(wumpus=(3,3), oro=(2,2), pozos=[(1,0), (3,1)])
state = W.reset()
W.render()
agente = Heroe()
agente.states.append(state)
print(agente.interp_percepto())

**Representando el mundo mediante reglas**

Ahora consideramos la información sobre la relación entre los pozos y la brisa. Observe que las reglas del mundo dicen que en las casillas adyacentes a un pozo se siente una brisa. De manera equivalente, se sigue que si el agente no siente una briza en una casilla, entonces no hay un pozo en las casillas adyacentes. Resulta natural representar esto como una implicación:

Si el agente está en $(0,0)$ y no siente una brisa, entonces no hay pozo en $(1,0)$ ni en $(0,1)$.

No obstante, vamos a ceñirnos a las limitaciones de la base de conocimiento. Esto es, vamos a usar solo el lenguaje de las reglas. Así pues, en lugar de una única fórmula, debemos incluir en la base de conocimiento las dos siguientes reglas:

* Si no percibo una brisa en $(0,0)$, no hay pozo en $(1,0)$ 
* Si no percibo una brisa en $(0,0)$, no hay pozo en $(0,1)$ 

Y esto para cada casilla:

* Si no percibo una brisa en $(1,0)$, no hay un pozo en $(0,0)$ 
* Si no percibo una brisa en $(1,0)$, no hay un pozo en $(2,0)$ 
* Si no percibo una brisa en $(1,0)$, no hay un pozo en $(1,1)$ 
* $\vdots$

La forma general de todas estas reglas es que si el agente está en la casilla $(x,y)$ y no siente una brisa, entonces no hay pozo en ninguna de sus casillas adyacentes.

In [ ]:
def brisa_pozo(self):
    formulas = []
    casillas = self.adyacentes(self.loc)
    for c in casillas:
        formulas += [
            f'-brisa{self.loc}>-pozo{c}',                
        ]
    return formulas

setattr(Heroe, 'brisa_pozo', brisa_pozo)

agente = Heroe()
agente.brisa_pozo()

<a class="anchor" id="ej4"></a>**Ejercicio 4:** 

([Anterior ejercicio](#ej3)) ([Próximo ejercicio](#ej5))

Escriba un código que genere una lista de fórmulas `hedor_wumpus` que contenga todas las implicaciones sobre la relación entre el Wumpus y el hedor dependiendo de la casilla en que se encuentra el agente.

Si el agente está en la casilla $(0,0)$, la respuesta debe ser:

```
['-hedor(0, 0)>-wumpus(1, 0)', '-hedor(0, 0)>-wumpus(0, 1)']
```

In [ ]:
def hedor_wumpus(self):
    formulas = []
    # AQUÍ COMENZA SU CÓDIGO
    
    # AQUÍ TERMINA SU CÓDIGO
    return formulas

setattr(Heroe, 'hedor_wumpus', hedor_wumpus)

agente = Heroe()
agente.hedor_wumpus()

---

<a class="anchor" id="ej5"></a>**Ejercicio 5:** 

([Anterior ejercicio](#ej4)) ([Próximo ejercicio](#ej6))

Escriba un código que genere una lista de fórmulas `casilla_segura`, la cual debe contener todas reglas que dicen que si en una casilla no hay ni pozo ni Wumpus, entonces es segura. (Vamos a ignorar por el momento el hecho de que la casilla sería segura incluso habiendo hedor, porque el Wumpus podría no estar vivo.)

* Si no hay ni pozo ni Wumpus en $(0,0)$, entonces $(0,0)$ es segura.
* $\vdots$

La respuesta debe ser:

```
['-pozo(0, 0)&-wumpus(0, 0)>segura(0, 0)',
 '-pozo(0, 1)&-wumpus(0, 1)>segura(0, 1)',
 '-pozo(0, 2)&-wumpus(0, 2)>segura(0, 2)',
 '-pozo(0, 3)&-wumpus(0, 3)>segura(0, 3)',
 '-pozo(1, 0)&-wumpus(1, 0)>segura(1, 0)',
 '-pozo(1, 1)&-wumpus(1, 1)>segura(1, 1)',
 '-pozo(1, 2)&-wumpus(1, 2)>segura(1, 2)',
 '-pozo(1, 3)&-wumpus(1, 3)>segura(1, 3)',
 '-pozo(2, 0)&-wumpus(2, 0)>segura(2, 0)',
 '-pozo(2, 1)&-wumpus(2, 1)>segura(2, 1)',
 '-pozo(2, 2)&-wumpus(2, 2)>segura(2, 2)',
 '-pozo(2, 3)&-wumpus(2, 3)>segura(2, 3)',
 '-pozo(3, 0)&-wumpus(3, 0)>segura(3, 0)',
 '-pozo(3, 1)&-wumpus(3, 1)>segura(3, 1)',
 '-pozo(3, 2)&-wumpus(3, 2)>segura(3, 2)',
 '-pozo(3, 3)&-wumpus(3, 3)>segura(3, 3)']
```

In [ ]:
def casilla_segura(self):
    formulas = []
    # AQUÍ COMENZA SU CÓDIGO
    
    # AQUÍ TERMINA SU CÓDIGO
    return formulas

setattr(Heroe, 'casilla_segura', casilla_segura)

agente = Heroe()
agente.casilla_segura()

---

# Inferencias y queries <a class="anchor" id="queries"></a>

([Volver al inicio](#inicio))

Ya tenemos todo lo que necesitamos para crear una base de conocimiento inicial. Con esta base podemos implementar un programa de agente que permita transitar por todas las casillas seguras. 

Comenzamos por inicializar la base de conocimiento así:

* Los datos serán la percepción sensorial, la posición y dirección actual del agente.
* Las reglas son la dinámica del mundo que hemos codificado hasta ahora. Estas reglas incluyen:
    - `brisa_pozo`
    - `hedor_wumpus`
    - `casilla_segura`

In [ ]:
W = Wumpus(wumpus=(0,2), oro=(3,2), pozos=[(2,0)])
W.render()
agente = Heroe()
state = W.reset()
agente.states.append(state)
c = agente.interp_percepto()
for hecho in c.split('&'):
    agente.base.tell(hecho)
print("¡Base de conocimiento creada!")
print(agente.base)

Además, observe que podemos utilizar el método `ask` de la clase `LPQuery` para hacer la consulta a la base de conocimiento acerca de si una casilla es segura o no:

In [ ]:
objetivo = 'segura(1, 0)'
print("Objetivo:", objetivo)
print()
print("Datos:", agente.base.hechos)
print()
print("Reglas aplicables para el objetivo:")
reglas_objeto = agente.base.reglas_aplicables(objetivo)
for r in reglas_objeto:
    print(' & '.join(r.antecedente), ">", r.consecuente)
res = agente.base.ask(objetivo)
print()
print("Resultado de la consulta:", res)

<a class="anchor" id="ej6"></a>**Ejercicio 6:** 

([Anterior ejercicio](#ej5))

Cree un código que implemente el método `agente.adyacentes_seguras()` que devuelva una lista con las casillas adyacentes seguras de acuerdo a la base de conocimiento del agente.

In [ ]:
def adyacentes_seguras(self):
    casillas_seguras = []
    # AQUÍ COMIENZA SU CÓDIGO
    
    # AQUÍ TERMINA SU CÓDIGO
    return casillas_seguras

setattr(Heroe,"adyacentes_seguras",adyacentes_seguras)

Pruebe su código con la siguiente celda:

In [ ]:
W = Wumpus(wumpus=(3,3), pozos=[(2,0), (2,2)])
state = W.reset()
agente = Heroe()
agente.states.append(state)
c = agente.interp_percepto()
for hecho in c.split('&'):
    agente.base.tell(hecho)
adyacentes_seguras = agente.adyacentes_seguras()
assert(adyacentes_seguras == [(1, 0), (0, 1)]), '¡Primer test Fallido!'
W = Wumpus(wumpus=(1,0), oro=(3,2), pozos=[(2,0), (2,2)])
state = W.reset()
agente = Heroe()
agente.states.append(state)
c = agente.interp_percepto()
for hecho in c.split('&'):
    agente.base.tell(hecho)
adyacentes_seguras = agente.adyacentes_seguras()
assert(adyacentes_seguras == []), '¡Segundo test Fallido!'
# Create environment
W = Wumpus(wumpus=(3,3), oro=(2,2), pozos=[(2,0), (3,1)])
# Create agent
agente = Heroe()
# Create episode
episodio = Episode(environment=W,\
        agent=agente,\
        model_name='Baseline',\
        num_rounds=15)
# Include plan
camino_ida = [(0,0), (0,1), (1,1)]
acciones = agente.acciones_camino(camino_ida)
agente.plan += acciones
episodio.run()
adyacentes_seguras = agente.adyacentes_seguras()
assert(adyacentes_seguras == [(1, 3), (0, 2)]), '¡Tercer test Fallido!'
# Create environment
W = Wumpus(wumpus=(1,2), oro=(2,2), pozos=[(2,0), (3,1)])
# Create agent
agente = Heroe()
# Create episode
episodio = Episode(environment=W,\
        agent=agente,\
        model_name='Baseline',\
        num_rounds=15)
# Include plan
camino_ida = [(0,0), (0,1), (0,2)]
acciones = agente.acciones_camino(camino_ida)
agente.plan += acciones
episodio.run()
adyacentes_seguras = agente.adyacentes_seguras()
assert(adyacentes_seguras == [(1, 0), (0, 1)]), '¡Cuarto test Fallido!'
print('¡Tests superados!')

---

Ahora evaluamos al agente sobre el test suite para encontrar el promedio de la suma de las recompensas y el porcentaje de éxito en que el agente logra salir con el oro:

In [ ]:
# Create experiment
exp = Experiment(num_rounds=100, \
                 num_episodes=1)
# Create and load environment
exp.load_test_suite("test_suite_wumpus.json")
# Create agent
agente = Heroe()
# Run the experiment
exp.run_experiment(agents=[agente], \
                   names=['Baseline'], \
                   measures=['histogram'],\
                   verbose=0)

A partir de estos datos observamos lo siguiente. Todos los entornos terminan en menos de 100 rondas. Al revisar el histograma vemos que no hay episodios en los cuales la recompensa esté cercana a -1000, lo cual significa que el agente nunca muere. Podemos asegurar que se logró el objetivo de que el agente explore la cueva del Wumpus de manera segura. 

En cuanto a la recompensa promedio, es solo de 324 puntos. Al examinar el histograma, vemos que puntajes cerca de 1000 ocurren un poco menos del 30% de las veces, que es cuando el agente logra salir con el oro. El porcentaje exacto de extracción del oro se puede calcular así:

In [ ]:
df1 = exp.data.groupby(['environment', 'episode']).reward.sum().reset_index()
df1['extraction'] = df1['reward'] > 0
df1['extraction'].value_counts()
print('Porcentaje de extracción:', df1['extraction'].value_counts()[1] / sum(df1['extraction'].value_counts())*100)

El porcentaje de extracción es aproximadamente el 36%. Sería de esperar que mejoras adicionales logren hacer que el agente aumente este porcentaje de recuperación del oro.

---

### En este notebook usted aprendió

* El mundo del Wumpus.
* Uso de la lógica proposicional para representar las reglas del mundo del Wumpus.